# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

## Get Required Files from Drive

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [10]:
#Set your project path 
project_path = '/content/drive/My Drive/Colab Notebooks/NLP'

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [28]:
#Import libraries

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [21]:
#Unzipping the input file
!unzip -uq '/content/drive/My Drive/Colab Notebooks/NLP/Data-20200620T202035Z-001.zip' -d '/content/drive/My Drive/Colab Notebooks/NLP/Data'

In [38]:
# Reading the asked json file
sar=pd.read_json(project_path+'/Data/Sarcasm_Headlines_Dataset.json',lines=True)
sar.head(2)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [41]:
# Dropping artilce_link field from the dataset
sar=sar.drop('article_link',axis=1)
sar.head(2)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [49]:
#Row wise length for headline
length_sar=sar['headline'].str.len()
length_sar.head(2)

0    78
1    84
Name: headline, dtype: int64

In [56]:
#Maximum length of a given headline 
max_len=length_sar.max()
max_len

254

#**## Modelling**

## Import required modules required for modelling.

In [57]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [58]:
max_features = 10000
maxlen = max_len
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [59]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(sar['headline'].values)

# Define X and y for your model.

In [60]:
X = tokenizer.texts_to_sequences(sar['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(sar['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [74]:
num_words=len(tokenizer.word_index)+1
num_words

29657

#**## Word Embedding**

## Get Glove Word Embeddings

In [70]:
glove_file = project_path + '/Data/glove.6B.zip'

In [71]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [72]:
EMBEDDING_FILE = project_path+'/Data/glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [75]:
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [ ]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

In [95]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(num_words, embedding_size,weights = [embedding_matrix],input_length = X.shape[1]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 254, 200)          5931400   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 254, 256)          336896    
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               355152    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 197       
Total params: 6,623,645
Trainable params: 6,623,645
Non-trainable params: 0
_________________________________________________________________
None


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [211]:
batch_size = 100
epochs = 5

In [225]:
# Splitting the data in train and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(21367, 254) (21367,)
(5342, 254) (5342,)


In [226]:
Y_test[0]

1

In [227]:
#training the model
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size)

Epoch 1/5
214/214 [==============================] - 319s 1s/step - loss: 0.0412 - accuracy: 0.9859
Epoch 2/5
214/214 [==============================] - 321s 1s/step - loss: 0.0262 - accuracy: 0.9912
Epoch 3/5
214/214 [==============================] - 317s 1s/step - loss: 0.0218 - accuracy: 0.9932
Epoch 4/5
214/214 [==============================] - 328s 2s/step - loss: 0.0199 - accuracy: 0.9939
Epoch 5/5
214/214 [==============================] - 330s 2s/step - loss: 0.0135 - accuracy: 0.9956


In [228]:
# Validation(Test)_ Accuracy
score,acc = model.evaluate(X_test, Y_test, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

54/54 [==============================] - 5s 99ms/step - loss: 0.6509 - accuracy: 0.8845
score: 0.65
acc: 0.88
